In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.models import Model,load_model, Sequential
from tensorflow.keras.layers import  GlobalAveragePooling2D, Dropout, Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import  Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
from tensorflow.keras import Model,layers
import glob
from tensorflow.keras.optimizers import RMSprop

In [2]:
glob.glob('D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train\*')

['D:\\Programming\\Python\\Kaggle\\Intel_Image_Classification\\seg_train\\buildings',
 'D:\\Programming\\Python\\Kaggle\\Intel_Image_Classification\\seg_train\\forest',
 'D:\\Programming\\Python\\Kaggle\\Intel_Image_Classification\\seg_train\\glacier',
 'D:\\Programming\\Python\\Kaggle\\Intel_Image_Classification\\seg_train\\mountain',
 'D:\\Programming\\Python\\Kaggle\\Intel_Image_Classification\\seg_train\\sea',
 'D:\\Programming\\Python\\Kaggle\\Intel_Image_Classification\\seg_train\\street']

In [3]:
def prepare_dataset(path,label):
    x_train=[]
    y_train=[]
    all_images_path=glob.glob(path+'/*.jpg')
    for img_path in all_images_path :
            img=load_img(img_path, target_size=(150,150))
            img=img_to_array(img)
            img=img/255.0
            x_train.append(img)
            y_train.append(label)
    return np.array(x_train),np.array(y_train)

In [4]:
paths=glob.glob('D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train\*')
l=len('D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train/')
labels=[]
for path in paths:
    labels.append(path[l:])
    print(labels)

['buildings']
['buildings', 'forest']
['buildings', 'forest', 'glacier']
['buildings', 'forest', 'glacier', 'mountain']
['buildings', 'forest', 'glacier', 'mountain', 'sea']
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [5]:
trainX_building, trainY_building  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train/buildings/",0)
trainX_forest,trainY_forest  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train/forest/",1)
trainX_glacier,trainY_glacier  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train\glacier/",2)
trainX_mount,trainY_mount  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train\mountain/",3)
trainX_sea,trainY_sea  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train\sea/",4)
trainX_street,trainY_street  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification\seg_train\street/",5)

print('train building shape ', trainX_building.shape, trainY_building.shape) 
print('train forest', trainX_forest.shape ,trainY_forest.shape)
print('train glacier', trainX_glacier.shape,trainY_glacier.shape)
print('train mountain', trainX_mount.shape, trainY_mount.shape)
print('train sea',     trainX_sea.shape, trainY_sea.shape)
print('train street', trainX_street.shape ,trainY_street.shape)

train building shape  (2191, 150, 150, 3) (2191,)
train forest (2271, 150, 150, 3) (2271,)
train glacier (2404, 150, 150, 3) (2404,)
train mountain (2512, 150, 150, 3) (2512,)
train sea (2274, 150, 150, 3) (2274,)
train street (2382, 150, 150, 3) (2382,)


In [6]:
x_train=np.concatenate((trainX_building,trainX_forest,trainX_glacier,trainX_mount,trainX_sea,trainX_street),axis=0)
y_train=np.concatenate((trainY_building,trainY_forest,trainY_glacier,trainY_mount,trainY_sea,trainY_street),axis=0)

In [7]:
print(x_train.shape)
print(y_train.shape)

(14034, 150, 150, 3)
(14034,)


In [8]:
from tensorflow.keras.applications import ResNet50

#step1
# file_resnet='/kaggle/input/vgg16/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrained_model=ResNet50( input_shape=(150,150,3),
                                  include_top=False,
                                  weights='imagenet'
                                   )
#step2
for layer in pretrained_model.layers:
     layer.trainable = False

# pretrained_model.summary()
        
#step3        
last_layer = pretrained_model.get_layer('conv5_block3_out')
print('last layer of vgg : output shape: ', last_layer.output_shape)
last_output = last_layer.output

#step4
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(6, activation='softmax')(x)

#step5
model_resnet = Model(pretrained_model.input, x) 

#step6
model_resnet.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

#step7
# model_resnet.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test))

last layer of vgg : output shape:  (None, 5, 5, 2048)


In [9]:
testX_building, testY_building  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification/seg_test/seg_test/buildings/",0)
testX_forest,testY_forest  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification/seg_test/seg_test/forest/",1)
testX_glacier,testY_glacier  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification/seg_test/seg_test/glacier/",2)
testX_mount,testY_mount  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification/seg_test/seg_test/mountain/",3)
testX_sea,testY_sea  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification/seg_test/seg_test/sea/",4)
testX_street,testY_street  = prepare_dataset("D:\Programming\Python\Kaggle\Intel_Image_Classification/seg_test/seg_test/street/",5)

x_test=np.concatenate((testX_building,testX_forest,testX_glacier,testX_mount,testX_sea,testX_street),axis=0)
y_test=np.concatenate((testY_building,testY_forest,testY_glacier,testY_mount,testY_sea,testY_street),axis=0)

In [10]:
model_resnet.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

Not enough GPU memory?